In [6]:
import os 
import cv2
import sys 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import sklearn 
import seaborn 
import torch 
import os.path as osp
from google.colab import drive

In [11]:
from sklearn.model_selection import train_test_split 

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html 

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

In [1]:
from mmdet.apis import init_detector, inference_detector 
import mmcv
import mmdet

In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

In [3]:
!mkdir /content/data

In [ ]:
!tar -xvf images.tar.gz -C /content/data 
!tar -xvf annotations.tar.gz -C /content/data

In [7]:
pet_df = pd.read_csv("./data/annotations/trainval.txt", sep=" ", header= None, names=["img_names", "class_id", "etc1", "etc2"] )

In [8]:
pet_df["class_name"] = pet_df["img_names"].apply(lambda x : x [:x.rfind("_")])

In [9]:
pet_df.head()

,img_names,class_id,etc1,etc2,class_name
0,Abyssinian_100,1,1,1,Abyssinian
1,Abyssinian_101,1,1,1,Abyssinian
2,Abyssinian_102,1,1,1,Abyssinian
3,Abyssinian_103,1,1,1,Abyssinian
4,Abyssinian_104,1,1,1,Abyssinian


In [12]:
train_df, val_df = train_test_split(pet_df, test_size=0.1, stratify=pet_df["class_id"], random_state=2022)

In [13]:
train_df['img_names'].to_csv('./data/train.txt', sep=' ', header=False, index=False)
val_df['img_names'].to_csv('./data/val.txt', sep=' ', header=False, index=False)

In [14]:
def show_version():
    print(f"========version========")
    print(f"python : {sys.version}")
    print(f"numpy : {np.__version__}")
    print(f"pandas : {pd.__version__}")
    print(f"cv2 : {cv2.__version__}")
    print(f"torch : {torch.__version__}")
    print(f"mmdet : {mmdet.__version__}")

def show_shape(dict_df):
    for k, v in dict_df.items():
        print(f"{k} {v.shape}")

  

In [15]:
dict_df = {"train_df":train_df , "val_df":val_df}
show_shape(dict_df)
show_version()

train_df (3312, 5)
val_df (368, 5)
========version========
python : 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
numpy : 1.21.6
pandas : 1.3.5
cv2 : 4.1.2
torch : 1.11.0+cu113
mmdet : 2.25.0


In [ ]:
pet_class_list = pet_df["class_name"].unique().tolist()
pet_class_list

In [17]:
import xml.etree.ElementTree as ET 
import mmcv
from mmdet.datasets.builder import DATASETS 
from mmdet.datasets.custom import CustomDataset

In [18]:
PET_CLASSES = pet_df["class_name"].unique().tolist()

In [19]:
def get_bounding_box_from_xml(anno_dir, xml_file):
    anno_xml_file = osp.join(anno_dir, xml_file)
    tree = ET.parse(anno_xml_file)
    root = tree.getroot()
    bbox_names = []
    bboxes = []

    for obj in root.findall("object"):
        bbox_name = xml_file[:xml_file.rfind("_")]

        xml_box = obj.find("bndbox")

        x1 = int(xml_box.find("xmin").text)
        y1 = int(xml_box.find("ymin").text)
        x2 = int(xml_box.find("xmax").text)
        y2 = int(xml_box.find("ymax").text)

        bboxes.append([x1, y1, x2, y2])
        bbox_names.append(bbox_name)

        return bbox_names, bboxes 

In [20]:
@DATASETS.register_module(force=True)

class PetDataset(CustomDataset):
    
    CLASSES = PET_CLASSES 
    
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
          

        data_infos = []

        for image_id in image_list :
            filename = "{0:}/{1:}.jpg".format(self.img_prefix, image_id)

            image = cv2.imread(filename)

            height, width = image.shape[:2]

            data_info = {
                "filename":str(image_id) + ".jpg", 
                "width":width, 
                "height":height
                }

            label_prefix = self.img_prefix.replace("images", "annotations")
            anno_xml_file = osp.join(label_prefix, "xmls/"+str(image_id)+".xml")

            if not osp.exists(anno_xml_file):
                 continue
            
            anno_dir = osp.join(label_prefix, "xmls")
            bbox_names, bboxes = get_bounding_box_from_xml(anno_dir, str(image_id)+".xml")

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = [] 

            for bbox_name, bbox in zip(bbox_names , bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else: 
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_igonre.append(-1)
                    
            data_anno = {
                "bboxes" : np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                "labels" : np.array(gt_labels, dtype=np.long),
                "bboxes_ignore" : np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
                "labels_ignore" : np.array(gt_labels_ignore, dtype=np.long)
            } 

            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        
        return data_infos



In [22]:
class PetDataset_Debug():
    CLASSES = PET_CLASSES
    def __init__(self, data_root, ann_file, img_prefix):
        self.data_root = data_root
        self.ann_file = osp.join(data_root, ann_file)
        self.img_prefix = osp.join(data_root, img_prefix)   
        self.data_infos = self.load_annotations(self.ann_file)

    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []

        for image_id in image_list :

            filename = "{0:}/{1:}.jpg".format(self.img_prefix, image_id)
            image = cv2.imread(filename)
            height, width = image.shape[:2]

            data_info = {"filename": str(image_id) + ".jpg", 
                         "width":width, 
                         "height":height}
            
            label_prefix = self.img_prefix.replace("images", "annotations")

            anno_xml_file = osp.join(label_prefix, "xmls/"+str(image_id)+".xml")
            if not osp.exists(anno_xml_file):
                continue
      

            anno_dir = osp.join(label_prefix, "xmls")
            bbox_names, bboxes = get_bounding_box_from_xml(anno_dir, str(image_id)+".xml")

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = [] 

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_lables_ignore.append(-1)
            
            data_anno = {
                'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                'labels': np.array(gt_labels, dtype=np.long),
                'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
                'labels_ignore': np.array(gt_labels_ignore, dtype=np.long)
                }

            data_info.update(ann=data_anno)

            data_infos.append(data_info)

        return data_infos

In [24]:
train_ds = PetDataset_Debug(data_root='/content/data', ann_file='train.txt', img_prefix='images')
print(train_ds.data_infos[:10])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

[{'filename': 'miniature_pinscher_157.jpg', 'width': 215, 'height': 300, 'ann': {'bboxes': array([[ 25.,  13., 200., 178.]], dtype=float32), 'labels': array([21]), 'bboxes_ignore': array([], shape=(0, 4), dtype=float32), 'labels_ignore': array([], dtype=int64)}}, {'filename': 'Bengal_173.jpg', 'width': 300, 'height': 182, 'ann': {'bboxes': array([[17., 11., 70., 67.]], dtype=float32), 'labels': array([5]), 'bboxes_ignore': array([], shape=(0, 4), dtype=float32), 'labels_ignore': array([], dtype=int64)}}, {'filename': 'Egyptian_Mau_133.jpg', 'width': 400, 'height': 300, 'ann': {'bboxes': array([[ 80.,   1., 144.,  78.]], dtype=float32), 'labels': array([11]), 'bboxes_ignore': array([], shape=(0, 4), dtype=float32), 'labels_ignore': array([], dtype=int64)}}, {'filename': 'Birman_121.jpg', 'width': 500, 'height': 333, 'ann': {'bboxes': array([[158.,  13., 413., 241.]], dtype=float32), 'labels': array([6]), 'bboxes_ignore': array([], shape=(0, 4), dtype=float32), 'labels_ignore': array([],

In [25]:
config_file = './mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = './mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

In [ ]:
!cd mmdetection; mkdir checkpoints
!wget -O ./mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

In [ ]:
from mmcv import Config 

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

In [28]:
drive.mount("/content/drive")

Mounted at /content/drive


In [29]:
!ln -s /content/drive/MyDrive/ /mydrive
!ls /mydrive

 ADSP.pdf	   'Deep Learning-1 - 선 차트 1.gsheet'   Mathematics	 Work
'Colab Notebooks'   수학.gdoc				  pet_work_dir	 試験
 dataset	    Git					  Presentation


In [30]:
!mkdir "/mydrive/pet_work_dir"

mkdir: cannot create directory ‘/mydrive/pet_work_dir’: File exists


In [31]:
!nvidia-smi

Mon Jul 11 07:38:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from mmdet.apis import set_random_seed


cfg.dataset_type = 'PetDataset'
cfg.data_root = '/content/data/'

cfg.data.train.type = 'PetDataset'
cfg.data.train.data_root = '/content/data/'
cfg.data.train.ann_file = 'train.txt'
cfg.data.train.img_prefix = 'images'

cfg.data.val.type = 'PetDataset'
cfg.data.val.data_root = '/content/data/'
cfg.data.val.ann_file = 'val.txt'
cfg.data.val.img_prefix = 'images'

cfg.model.roi_head.bbox_head.num_classes = 37
cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'


cfg.work_dir = '/mydrive/pet_work_dir'
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 5

cfg.runner.max_epochs = 5
cfg.evaluation.metric = 'mAP'
cfg.evaluation.interval = 5
cfg.checkpoint_config.interval = 5

cfg.data.samples_per_gpu = 4
cfg.seed = 0
cfg.device = "cuda"
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.lr_config.policy='step'
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# train용 Dataset 생성. 
datasets = [build_dataset(cfg.data.train)]

In [34]:
datasets

[
 PetDataset Train dataset with number of images 3304, and instance counts: 
 +-----------------------+-------+-------------------------+-------+-------------------------------+-------+---------------------+-------+---------------------------------+-------+
 | category              | count | category                | count | category                      | count | category            | count | category                        | count |
 +-----------------------+-------+-------------------------+-------+-------------------------------+-------+---------------------+-------+---------------------------------+-------+
 | 0 [Abyssinian]        | 89    | 1 [american_bulldog]    | 90    | 2 [american_pit_bull_terrier] | 90    | 3 [basset_hound]    | 90    | 4 [beagle]                      | 90    |
 | 5 [Bengal]            | 88    | 6 [Birman]              | 90    | 7 [Bombay]                    | 86    | 8 [boxer]           | 90    | 9 [British_Shorthair]           | 90    |
 | 10 [chihuahua]

In [35]:
%cd mmdetection

/content/mmdetection


In [36]:
model = build_detector(cfg.model, train_cfg=cfg.get("train_cfg"), test_cfg=cfg.get("test_cfg"))
model.CLASSES = datasets[0].CLASSES

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-07-11 07:38:59,744 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by i

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 367/367, 8.3 task/s, elapsed: 44s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-07-11 09:05:03,281 - mmdet - INFO - 
+----------------------------+-----+------+--------+-------+
| class                      | gts | dets | recall | ap    |
+----------------------------+-----+------+--------+-------+
| Abyssinian                 | 10  | 74   | 1.000  | 0.781 |
| american_bulldog           | 10  | 85   | 1.000  | 0.692 |
| american_pit_bull_terrier  | 10  | 46   | 0.800  | 0.601 |
| basset_hound               | 10  | 61   | 1.000  | 0.991 |
| beagle                     | 10  | 128  | 1.000  | 0.718 |
| Bengal                     | 10  | 84   | 1.000  | 0.406 |
| Birman                     | 10  | 63   | 1.000  | 0.564 |
| Bombay                     | 10  | 96   | 1.000  | 0.844 |
| boxer                      | 10  | 99   | 1.000  | 0.741 |
| British_Shorthair          | 10  | 55   | 1.000  | 0.765 |
| chihuahua                  | 10  | 78   | 1.000  | 0.935 |
| Egyptian_Mau               | 8   | 63   | 0.875  | 0.225 |
| english_cocker_spaniel     | 10  | 37   |